#Lab.07 / IBM3202 – Dinámica Molecular en GROMACS

### Aspectos teóricos

Como discutimos en clase, el mecanismo central de **dinámica molecular (MD)** es **resolver numéricamente la ecuación de movimiento de Newton** al derivar la **energía potencial** (es decir, la energía de las interacciones ligadas y no ligadas) para cada átomo durante la propagación de sus cambios de posición a lo largo del tiempo. Estos cambios de posición son consecuencia tanto de las **interacciones átomo-átomo** como de los **movimientos térmicos** (energía cinética).


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/MD_01.png'/>
<figcaption>FIGURA 1. El rendimiento (ns/día) de las simulaciones de dinámica molecular depende de (A) el número de átomos del sistema simulado y (B) el uso exclusivo de la CPU o combinado con nodos de GPU para realizar las simulaciones. <br> Kutzner C et al (2014) <i> IOS Press, 722-730</i>; Kutzner C et al (2019) <i> J Comput Chem 40(27), 2418-2431</i></figcaption></center>
</figure>

Dado que la mayor parte de la energía potencial es un cálculo de pares, con **la mayoría de estos cálculos correspondientes a interacciones no ligadas** (y que requieren funciones de cambio y estrategias de corte de distancia para reducir sus costos computacionales), el **tiempo necesario para resolver estas ecuaciones aumenta exponencialmente con el número de partículas en el sistema**. En años anteriores, estas necesidades computacionales se aliviaban típicamente con inversiones costosas en hardware de potencia de cálculo de CPU. Pero en los últimos años, se han desarrollado varios paquetes de simulación de MD para calcular la mayoría (si no todas) de las **interacciones no ligadas** en GPUs de grado consumidor, descargando la CPU y generando mejoras significativas de velocidad en computadoras asequibles.

Afortunadamente, con la llegada de la **computación en la nube** y el surgimiento de **Google Colab**, se depende en gran medida del uso de GPUs. Por lo tanto, estos servicios en la nube se pueden utilizar eficientemente para realizar simulaciones de MD de **10 a 100 ns de duración**.




##Aspectos Generales

Inspirados por la pandemia de COVID-19, en esta sesión de laboratorio realizaremos una simulación de dinámica molecular (MD) de la **proteasa tipo papaina de SARS-CoV-2**, un objetivo actual para el diseño de fármacos contra este virus.

Para nuestra sesión de laboratorio, compilaremos e instalaremos **GROMACS**, un paquete de simulación de MD que utilizaremos para configurar y llevar a cabo nuestras simulaciones. Visualizaremos la estructura de nuestra proteína utilizando **py3Dmol**, mientras que las trayectorias de simulación de nuestras ejecuciones de producción de MD se visualizarán en una versión web de **NGLview**. También analizaremos algunos parámetros relacionados con la equilibración adecuada de nuestro sistema de simulación, pero un análisis más profundo de los movimientos de la proteína se llevará a cabo en el próximo tutorial.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/MD_02.jpg' />
<figcaption>FIGURA 2. Pasos generales para realizar simulaciones de dinámica molecular (MD). Comenzando con una estructura de proteína experimental o modelada, se prepara un sistema de simulación configurando una caja de simulación, solvatando la proteína con agua y neutralizando la carga total del sistema con contraiones. Luego, se realizan las simulaciones, se visualizan y se analizan en términos de los movimientos de la proteína y sus características energéticas.
<br>Morgnanesi D et al (2015) <i>Antiviral Res 123, 204-215</i></figcaption></center>
</figure>

# Parte 0. Descarga e instalación del software necesario


Antes de comenzar, debes:
1. Recordar **iniciar el entorno de ejecución alojado** en Google Colab.
2. **¡MUY IMPORTANTE‼️** Ir a *Entorno de ejecución* -> *Cambiar tipo de entorno de ejecución* y seleccionar GPU.

Luego, debemos instalar tres piezas de software para realizar este tutorial, a saber:
- **biopython** para la manipulación de archivos PDB.
- **py3Dmol** para la visualización de la estructura de la proteína.
- **GROMACS** para preparar nuestro sistema de MD y realizar nuestras simulaciones de MD.

Para visualizar nuestras trayectorias de MD, emplearemos una versión web de **NGLview**. Esto se debe a la incapacidad de Google Colab para manejar un paquete de Python necesario para cargar NGLview directamente en Google Colab. Con suerte, esto cambiará en un futuro cercano. ¡El análisis de nuestras trayectorias se realizará principalmente en la próxima sesión de laboratorio!

1. Comenzaremos configurando **GROMACS** en Google Colab, basándonos en una instalación y compilación previas de GROMACS.


In [ ]:
# Descarga y descomprime GROMACS 2023.3
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs-2023.3.tar.gz
!tar xzf gromacs-2023.3.tar.gz

In [ ]:
# Revisando que funcione
%%bash
source /content/gromacs-2023/bin/GMXRC
gmx -h

2. Instalamos py3Dmol

In [ ]:
#Usando pip
!pip install py3Dmol

In [ ]:
#Importando py3Dmol
import py3Dmol

3. Instalamos **biopython**

In [ ]:
#Usando pip
!pip install biopython

Una vez completados estos procesos de instalación del software, estamos listos para realizar nuestros experimentos.


# Parte I – Configuración del sistema de simulación de MD


GROningen Machine for Chemical Simulations (**GROMACS**) es un software de código abierto y gratuito desarrollado por la Universidad de Groningen con un soporte consistente y continuo. También ha sido optimizado para cálculos maximizando el uso de todos los recursos computacionales disponibles (GPU y CPU). Una vez instalado, puedes acceder a todos los módulos que tiene para configurar, ejecutar y analizar simulaciones de MD simplemente ingresando:

**`gmx [módulo]`**

La lista de módulos se puede mostrar cuando **`[módulo]=help`**. ¡Inténtalo aquí!


In [ ]:
# Necesitaremos constantemente cargar GMXRC para que GROMACS funcione
%%bash
source /content/gromacs-2023/bin/GMXRC

# ¡Prueba gmx aquí!


Los pasos iniciales para configurar un sistema para simulaciones de MD son:

1. Realizar una **limpieza** de las coordenadas atómicas de entrada.
2. **Parametrizar** los átomos que conforman nuestro sistema.
3. **Solvatar** nuestra proteína (y agregar una membrana lipídica en el caso de proteínas de membrana).
4. **Agregar** contraiones para neutralizar la carga global del sistema.


### Parte I.1 – Limpieza de las coordenadas atómicas de entrada


Para este paso, primero necesitamos un conjunto inicial de coordenadas atómicas, generalmente provenientes de una estructura de proteína descargada del Banco de Datos de Proteínas (PDB). Una vez descargados, estos archivos PDB deben limpiarse de moléculas de agua (en el caso de estructuras cristalinas) o debe seleccionarse un modelo único de entre muchas soluciones para un conjunto único de desplazamientos químicos (en el caso de estructuras de RMN).

1. Comenzaremos haciendo una carpeta para preparar nuestro sistema y ejecutar nuestras simulaciones, como hemos hecho en el pasado (¿recuerdas nuestro tutorial de Acoplamiento Molecular?).


In [ ]:
# Primero hagamos una carpeta. Necesitamos importar las bibliotecas os y path
import os
from pathlib import Path

# Luego, definimos la ruta de la carpeta que queremos crear.
# Observa que la carpeta HOME para un entorno de ejecución alojado en Colab es /content/
mdpath = Path("/content/md01/")

# Ahora, creamos la carpeta utilizando el comando os.mkdir()
# La condición if es solo para verificar si la carpeta ya existe
# En caso afirmativo, Python devuelve un error
if os.path.exists(mdpath):
  print("La ruta ya existe")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("La ruta se creó correctamente")


2. Luego, cambiaremos al directorio de esta carpeta recién creada y descargaremos la estructura recientemente resuelta por rayos X de la proteasa tipo papaina (PLpro) de SARS-CoV-2 (PDB 6WZU).


In [ ]:
# Primero, cambiaremos al nuevo directorio. ¡Usaremos Python ahora! :)
os.chdir(mdpath)

In [ ]:
#Importa el PDB con biopython
import os
from Bio.PDB import *
pdbid = ['6wzu']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

3. Esta estructura tiene algunos residuos en dos conformaciones diferentes (¿recuerdas cuando discutimos la **ocupación** en clases y durante nuestro segundo tutorial?). Podemos filtrar uno de ellos.


In [ ]:
# Aquí configuramos un analizador para nuestro PDB
parser = PDBParser()
io = PDBIO()
estructura = parser.get_structure('X', '6wzu.pdb')
# Y aquí configuramos la conformación del residuo que queremos conservar
keepAltID = "A"

class KeepOneConfOnly(Select):  # Hereda métodos de la clase Select
    def accept_atom(self, atom):
        if (not atom.is_disordered()) or atom.get_altloc() == keepAltID:
            atom.set_altloc(" ")  # Elimina el ID de la ubicación alternativa antes de la salida.
            return True
        else:  # La ubicación alternativa no era la que se va a guardar.
            return False
        # fin de accept_atom()

# Esto mantendrá solo una conformación para cada residuo
io.set_structure(estructura)
io.save("6wzu_ready.pdb", select=KeepOneConfOnly())
print("Tu archivo PDB fue procesado. Se eliminaron las conformaciones alternativas de las cadenas laterales.")

4. Comúnmente, las moléculas de agua cristalográficas y otras moléculas de solvente se eliminan de la estructura resuelta experimentalmente antes de preparar el sistema de MD. Esto también es útil para eliminar hidrógenos, que a menudo se encuentran en estructuras de RMN, ya que son difíciles de manejar con las convenciones atómicas de un campo de fuerza dado. Podemos hacer esto usando `Dice` de **biopython**, como se muestra en la celda de código a continuación.

❗️**NOTA:** Este procedimiento no es universalmente apropiado. Algunas moléculas de agua podrían ser esenciales para la función de una proteína y, por lo tanto, se mantendrían en la simulación. Además, `Dice` elimina los ligandos, que en algunos casos deben mantenerse dependiendo del objetivo de tus simulaciones y de la capacidad para parametrizar tus ligandos. A veces, será más útil usar `grep`. Por ejemplo:


  `!grep -v "HOH" 6WZU_ready.pdb > 6WZU_clean.pdb`
  
  Este comando grep eliminaría solo las moléculas de agua del archivo PDB.

In [ ]:
# Aquí configuramos un analizador para nuestro PDB
parser = PDBParser()
io = PDBIO()
estructura = parser.get_structure('X', '6wzu_ready.pdb')
# Y aquí eliminamos hidrógenos, moléculas de agua y ligandos usando Dice
io.set_structure(estructura)
sel = Dice.ChainSelector('A', 1, 5000)
io.save("6wzu_clean.pdb", sel)
print("Tu archivo PDB fue procesado. Solo se conservaron los átomos pesados de la proteína.")


5. Carguemos la proteína con la que estamos trabajando usando py3dmol


In [ ]:
# Primero asignamos py3Dmol.view como view
view = py3Dmol.view()

# Las siguientes líneas se utilizan para agregar la clase addModel
# para leer archivos PDB
view.addModel(open('6wzu_clean.pdb', 'r').read(), 'pdb')

# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')

# Aquí establecemos el estilo de visualización y color
view.setStyle({'chain':'A'}, {'cartoon': {'color':'spectrum'}})

# Centramos la vista en todos los átomos visibles
view.zoomTo()

# Finalmente, visualizamos las estructuras con el comando a continuación
view.show()

### Parte I.2 – Parametrización de los átomos que componen nuestro sistema


Ahora, trabajaremos con GROMACS para parametrizar nuestra proteína, generando:

*   Un archivo de coordenadas **.gro** o **.pdb** que contiene todos los tipos de átomos según lo definido por un campo de fuerza dado (incluyendo hidrógenos).
*   Un archivo de topología **.top** que contiene los parámetros para enlaces, ángulos, torsiones e interacciones no enlazadas definidas por un campo de fuerza dado (función de energía potencial) para utilizar en nuestras simulaciones.

1. Parametrizaremos nuestra proteína utilizando el campo de fuerza **AMBER99SB-ILDN** en GROMACS y obtendremos estos archivos utilizando `gmx`, como se muestra en la celda de código a continuación. Este campo de fuerza se utiliza ampliamente en simulaciones de dinámica molecular y tiene parámetros que representan bien la dinámica y flexibilidad de proteínas plegadas. Ten en cuenta que la dinámica de proteínas altamente móviles o regiones intrínsecamente desordenadas no es el conjunto principal de datos para el cual se parametrizó este campo de fuerza, y puede haber otras opciones más adecuadas para tales objetivos.


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

#Utilizando pdb2gmx para parametrizar nuestro archivo PDB con el campo de fuerza AMBER y agua SPC/E
gmx pdb2gmx -f 6wzu_clean.pdb -o 6wzu_processed.pdb -water spce -ignh -ff amber99sb-ildn

Cuando utilizamos este comando, la opción `-f` corresponde al archivo de entrada, la opción `-ff` permite definir el campo de fuerza a utilizar y la opción `-o` proporciona el nombre de salida para el archivo de coordenadas (.coordinate). El archivo de topología recibe el nombre predeterminado `topol.top`, aunque esto se puede modificar con la opción `-p`.

También estamos indicando el tipo de molécula de agua que utilizaremos en nuestras simulaciones a través de la opción `-water`. Sí, hay muchos modelos de agua, pero sus características están fuera del alcance de nuestro tutorial. Nuestro modelo de agua seleccionado es SPC/E (carga puntual simple - extendida), que modela las moléculas de agua como una molécula de 3 puntos, cada una con su propia carga. TIP3P es un modelo de agua más común, similar a SPC/E pero con parámetros ligeramente diferentes. La precisión del modelo utilizado cambia según los cálculos que realizarás, ya que su dinámica y energética serán diferentes.

❗️**NOTA:** Cuando trabajas con archivos NMR, es útil incorporar la opción `-ignh`, que permite ignorar los hidrógenos contenidos en el archivo PDB. Durante la parametrización, los hidrógenos siempre se añaden de nuevo por el campo de fuerza, ya que contiene los parámetros de distancia típicos para todos los átomos basados en análisis experimental o de mecánica cuántica.

### Parte I.3 – Solvatando la proteína

Ahora definiremos una caja periódica para nuestro sistema de simulación, en la cual nuestra proteína estará centrada, y luego llenaremos esta caja con moléculas de agua, logrando así la solvatación de nuestra proteína.

Dado el uso de **condiciones de contorno periódicas** y los costos computacionales de la evaluación de interacciones no enlazadas, es imperativo definir adecuadamente la distancia de la caja periódica de manera que sea lo suficientemente grande como para evitar interacciones entre nuestra molécula y sus imágenes periódicas, al mismo tiempo que se minimiza el número de moléculas de agua (en esencia, el número de interacciones no enlazadas a evaluar durante la simulación).

A menudo se recomienda, para proteínas globulares, tener una **distancia de relleno** entre el átomo de la proteína más alejado del centro de la caja periódica y sus bordes de 1.0 a 1.5 nm.
Si tu sistema tiene una dimensión mucho más grande que las otras (por ejemplo, proteínas fibrilares), es posible que desees dibujar esta caja periódica con más cuidado.

1. En primer lugar, generaremos una caja periódica utilizando `editconf` de la siguiente manera:


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

#Ejecutar editconf para crear una caja cúbica con un relleno de 1.0 nm
gmx editconf -f 6wzu_processed.pdb -o 6wzu_newbox.pdb -c -d 1.0 -bt cubic

Las opciones proporcionadas a `editconf` permiten centrar (`-c`) las coordenadas de los átomos de nuestra proteína (`-f`) en una caja cúbica (`-bt cubic`) con una distancia de relleno de 1.0 nm desde el borde de la caja (`-d 1.0`).

Dadas las condiciones de contorno periódicas, esto significa que el sistema estará a una distancia mínima de 2.0 nm de sí mismo, en la cual la mayoría de los términos no enlazados están fuera de rango entre las imágenes periódicas debido al uso de umbrales de distancia. Por lo tanto, las únicas moléculas que interactuarán entre los bordes de la caja periódica son las moléculas de agua e iones. Estas nuevas coordenadas y la caja de simulación se guardan en un nuevo archivo de coordenadas (`-o`).


2. Una vez que la caja periódica está definida, procedemos a llenar esta caja con moléculas de agua utilizando `solvate`. Para hacerlo, ejecutamos:


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

#Usando solvate para añadir moléculas de agua
gmx solvate -cp 6wzu_newbox.pdb -o 6wzu_solv.pdb -p topol.top

Por favor, ten en cuenta que, dado que estamos añadiendo moléculas de agua a nuestro sistema de simulación, estamos generando **un nuevo archivo de topología** (opción `-p`) y **un nuevo archivo de coordenadas con las moléculas de agua añadidas** (opción `-o`).

Cada vez que escribimos un nuevo archivo de topología (o cualquier archivo con el mismo nombre), GROMACS realiza una copia de seguridad del archivo anterior añadiendo un símbolo `#` (es decir, `#topol.top.1`) y luego genera el nuevo archivo .top. De esta manera, no es necesario preocuparse por hacer copias de seguridad de los archivos.


**PREGUNTA❓:** A partir de la lectura de la salida del comando anterior, ¿cuántas moléculas de agua se añadieron?


3. ¡Veamos el sistema solvatado utilizando py3Dmol! ¿Cómo están solvatando las moléculas de agua a la proteína? ¿Están distribuidas homogéneamente las moléculas?


In [ ]:
# Primero asignamos py3Dmol.view como view
view = py3Dmol.view()
# Las siguientes líneas se utilizan para agregar la clase addModel
# para leer archivos PDB
view.addModel(open('6wzu_solv.pdb', 'r').read(), 'pdb')
# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')
# Aquí configuramos el estilo de visualización y el color
view.setStyle({'cartoon': {'color': 'green'}})
# Aquí agregamos un estilo para mostrar el oxígeno de las moléculas de agua
view.addStyle({'atom': 'OW'}, {'sphere': {'radius': '0.2'}})
# Centramos la vista en todos los átomos visibles
view.zoomTo()
# Y finalmente visualizamos las estructuras con el siguiente comando
view.show()

## Parte I.4 – Añadiendo contraiones para neutralizar la carga global del sistema

Ahora tenemos una caja solvatada, pero nuestro sistema tiene una **carga no nula**. Esto es un problema, ya que cada caja periódica tendrá una carga formal, lo que podría provocar que los potenciales electrostáticos sean irrealmente altos entre las imágenes periódicas. Por lo tanto, necesitamos neutralizar las cargas de nuestro sistema de simulación.

1. Determinemos fácilmente la carga absoluta en nuestro sistema utilizando `grep`.

In [ ]:
!grep "qtot" topol.top

**Pregunta❓:** Cuál es la carga total del sistema?

Para neutralizar el sistema de simulación, reemplazaremos las moléculas de agua con los contraiones necesarios para que la carga absoluta del sistema sea **cero**. Aunque esto puede ser suficiente para proteínas globulares en agua, en algunos casos podría ser más útil alcanzar una concentración de sal específica o generar diferencias de carga, por ejemplo, en simulaciones de canales iónicos en una bicapa lipídica.


2. La adición de contraiones se logra construyendo un **archivo de entrada binario ejecutable portátil para GROMACS**, o archivo **.tpr**. Este archivo contiene la información de las coordenadas, la topología/parámetros y un **archivo de instrucciones de dinámica molecular** (**.mdp**). Este archivo de instrucciones contiene todos los parámetros que se utilizarán para ejecutar diferentes cálculos, y puedes ingresar banderas que serán leídas por programas específicos.

  Descargaremos y utilizaremos un archivo **ions.mdp** predefinido (**¡échale un vistazo!**) en nuestra carpeta de simulación y generaremos el archivo de entrada de ejecución .tpr ejecutando el siguiente comando con el módulo `grompp`:


In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/ions.mdp
source /content/gromacs-2023/bin/GMXRC

#Utilizando grompp y un archivo de instrucciones de dinámica molecular para añadir contraiones a nuestro sistema
gmx grompp -f ions.mdp -c 6wzu_solv.pdb -p topol.top -o ions.tpr

Por favor ten en cuenta que, en el caso de `grompp`, la opción `-f` se utiliza ahora para el archivo .mdp, mientras que `-c` se utiliza para las coordenadas.

3. Una vez hecho esto, ejecuta el módulo `genion` para reemplazar aleatoriamente moléculas de agua en tu sistema de simulación con un número y tipo apropiado de contraiones.

❗️**NOTA:** Por lo general, `gmx` **nos pediría interactivamente** que elijamos un grupo para este reemplazo (en nuestro caso, el grupo 13 "SOL", correspondiente a las moléculas de agua). Aquí, estamos resolviendo la limitación de hacer esto en Google Colab al **generar un archivo de texto con estas opciones**, que luego pasamos a `gmx` para que lo lea.

In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

# Este es un truco para proporcionar opciones interactivas a gmx
echo "SOL" > options
echo " " >> options

# Utilizando genion y el archivo tpr para añadir contraiones a nuestro sistema solvatado
gmx genion -s ions.tpr -o 6wzu_solv_ions.pdb -p topol.top -pname NA -nname CL -neutral < options

**Pregunta❓:** Cuál es el tipo de iones monovalentes que añadimos? cuantos iones añadimos?

Hemos finalizado la preparación del sistema de simulación con nuestra proteína en el centro de una caja cúbica con un relleno de 1.0 nm, solvatada en moléculas de agua y neutralizada con contraiones. ¡Puedes optar por visualizarlo utilizando py3Dmol si lo deseas!


#Parte II – Minimización y Equilibrado del Sistema de Dinámica Molecular

Ahora, estamos listos para realizar la minimización de nuestro sistema para eliminar la alta energía y las fuerzas debido a malas coordenadas iniciales, así como su equilibrado a presión y temperatura constantes (conjunto NPT).

1. Comenzaremos descargando un archivo de instrucciones de dinámica molecular que contiene todos los parámetros necesarios para la minimización de nuestro sistema. También imprimiremos su contenido para inspeccionarlo.

In [ ]:
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/em.mdp


In [ ]:
#Revisemos el contenido del archivo MDP
!paste em.mdp

Como puedes ver, el proceso de minimización implica el uso de un integrador de **descenso más pronunciado** (`steep`). Este integrador, junto con el método de gradiente conjugado (`cg`) y un método cuasi-newtoniano (`l-bfgs`), son **algoritmos de minimización** que, en lugar de resolver las posiciones para cambios en el gradiente (ecuación de movimiento de Newton), buscan cambios en la posición que **minimizarían la energía potencial**.

Estos protocolos de minimización (a veces llamados relajación) se aseguran de que el **punto de inicio de nuestras simulaciones tenga un ΔU lo suficientemente favorable** para que los algoritmos de Verlet y leapfrog (que permiten la **propagación de cambios en las posiciones de los átomos para cada paso de tiempo**) sean numéricamente integrables.

En palabras más sencillas, este paso es crítico, ya que comenzar desde una alta energía potencial puede generar desde simulaciones altamente divergentes, en los casos más leves, hasta inestabilidades energéticas que harían que nuestro sistema sea imposible de integrar numéricamente (típicamente conocido como un **"sistema que explota"**).


2. Preprocesaremos nuestros archivos con `grompp` y luego ejecutaremos nuestra dinámica molecular con `mdrun`. Compara la similitud (y también las diferencias) en el uso de `grompp` para añadir contraiones neutralizantes y para la minimización del sistema.


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

# Utilizando grompp para preparar nuestra dinámica molecular de minimización
gmx grompp -f em.mdp -c 6wzu_solv_ions.pdb -p topol.top -o em.tpr

# Ejecutamos nuestra minimización
gmx mdrun -v -deffnm em -nb gpu

Para `mdrun`, estamos utilizando la opción de verborrea (`-v`) para mostrar los cambios en las fuerzas entre átomos durante la minimización del sistema. A medida que el sistema se minimiza, estas fuerzas deberían reducirse en comparación con el sistema inicial.

La opción `-deffnm` simplemente indica que todos los archivos generados por la dinámica molecular tendrán el mismo nombre estándar (en este caso, *em*) que el archivo .tpr generado con `grompp`.

**📚TAREA:** Investiga qué archivos se han generado después de `mdrun`. ¿Qué contiene cada archivo generado? Si no lo sabes, **¡búscalo en Google!** Este paso es **obligatorio**.


😱 **¡COPIA DE SEGURIDAD DE EMERGENCIA!** ¿Te quedaste sin tiempo y no pudiste minimizar tu sistema? Utiliza la siguiente celda de código para descargar un sistema ya minimizado:


In [ ]:
#Usar sólo en caso de emergencia
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab07/6wzu_em.tar.gz
!tar xzf 6wzu_em.tar.gz

3. Una vez que nuestra minimización haya terminado, podemos verificar cómo cambia la energía potencial del sistema en cada paso de minimización. Para ello, utilizaremos el módulo `energy` para extraer la energía potencial y luego la representaremos gráficamente utilizando **matplotlib**:


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

# Este es un truco para proporcionar opciones interactivas a gmx
echo "Potential" > options
echo " " >> options

# Utilizando energy para extraer la energía potencial del sistema
gmx energy -f em.edr -o em_potential.xvg -xvg none < options

In [ ]:
# Graficando la energía potencial del sistema
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

# Leyendo el archivo de texto que contiene esta información
data = np.loadtxt('em_potential.xvg')

plt.title('Potential Energy during Minimization')
plt.xlabel('Energy Minimization Step')
plt.ylabel(r'E$_P$ [kJ•mol$^{-1}]$')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

A continuación, comenzaremos nuestra **equilibración**. Este paso es necesario dado que optimizamos las posiciones de los átomos según nuestro campo de fuerza, pero en ningún momento maximizamos las interacciones entre el disolvente y el soluto. Tampoco hemos establecido el **efecto cinético de la temperatura sobre los movimientos de los átomos** o la **presión de nuestro sistema**, lo que lo hace inconexo con las condiciones experimentales.

Por lo tanto, **equilibraremos la energía y la densidad de nuestro sistema a temperatura y presión constantes** antes de las ejecuciones de producción de la dinámica molecular (MD).

4. El primer paso de equilibración es la **temperatura**. Por lo tanto, comenzaremos una simulación de dinámica molecular (MD) para equilibrar el sistema a una temperatura objetivo (en nuestro caso, 300K) utilizando un baño térmico. Las velocidades iniciales para los átomos de nuestro sistema a la temperatura objetivo se obtienen a través de una distribución de Maxwell.

  Primero descargaremos un archivo de instrucciones de MD adecuado y luego ejecutaremos nuestra simulación como lo hicimos antes para la minimización. También ten en cuenta que ahora estamos utilizando **recursos de GPU** para las interacciones no enlazadas.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/nvt.mdp

# Utilizando grompp para preparar nuestra dinámica molecular de equilibración NVT
source /content/gromacs-2023/bin/GMXRC
gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
%%time
%%bash
#Ejecutar nuestra dinámica molecular de equilibración NVT
source /content/gromacs-2023/bin/GMXRC
gmx mdrun -deffnm nvt -nb gpu

😱 **¡COPIA DE SEGURIDAD DE EMERGENCIA!** ¿Te quedaste sin tiempo y no pudiste realizar la equilibración NVT de tu sistema? Utiliza la siguiente celda de código para descargar un sistema ya equilibrado a temperatura:

In [ ]:
#Sólo en caso de emergencia
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab07/6wzu_nvt.tar.gz
!tar xzf 6wzu_nvt.tar.gz

Si observas el archivo de instrucciones de dinámica molecular (MD) para este paso, notarás que ahora estamos utilizando un **algoritmo de leapfrog** (`md`) para la propagación de los movimientos de la proteína a lo largo del tiempo.

Entre varios parámetros, podemos ver que los **átomos de hidrógeno están restringidos**. Esto se debe a que esos núcleos tienen poca masa y suelen estar dentro del radio de Lennard-Jones de una molécula, carecen de cualquier tipo de movimiento diédrico, y la distancia del enlace se conserva en gran medida. Es por eso que se utilizan algoritmos de restricción para simplificar las ecuaciones de movimiento, al tiempo que se conserva el efecto que tienen en la solvatación e interacciones. Dado que los movimientos internos más rápidos corresponden a las vibraciones de los enlaces entre átomos ligeros (como el hidrógeno), su restricción permite el uso de **timesteps más largos** (en nuestro caso, utilizaremos **2 fs**).

Finalmente, en este paso de equilibración se utiliza un **algoritmo de restricción de posición** para mantener en su lugar los átomos pesados de nuestra proteína. Esto se debe a que conocemos que la estructura de nuestra proteína es en su mayoría ideal (porque se modela a partir de datos experimentales), pero nuestras moléculas de disolvente se dibujaron alrededor de ella. Por lo tanto, en este paso mantendremos nuestra proteína en su mayoría en su lugar y permitiremos que el disolvente se reorganice alrededor de nuestra proteína como esperaríamos que lo haga a 300 K. Esta información está en el archivo **posre.itp** (un archivo de restricción de posición) que se generó al principio del uso de GROMACS. Esta restricción se lleva a cabo mediante la opción `-r em.gro`.


5. Lo que acabamos de hacer corresponde a una configuración de simulación en la que el número de átomos, el volumen y la temperatura se mantienen constantes: **conjunto NVT**. Por lo tanto, la temperatura del sistema debería oscilar alrededor de la temperatura deseada (en nuestro caso, 300 K). ¡Verifiquemos si esto es cierto!


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

# Este es un truco para proporcionar opciones interactivas a gmx
echo "Temperature" > options
echo " " >> options

# Utilizando energy para extraer la temperatura del sistema durante la equilibración NVT
gmx energy -f nvt.edr -o nvt_temp.xvg -xvg none < options

In [ ]:
# Graficando la temperatura del sistema
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

# Leyendo el archivo de texto que contiene esta información
data = np.loadtxt('nvt_temp.xvg')

plt.title('Temperature during 0.1 ns Equilibration (NVT)')
plt.xlabel('Time (ps)')
plt.ylabel('Temperature [K]')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

6. El paso final de equilibración nos permitirá implementar la **regulación de la presión** en nuestro sistema, lo que mantendrá constante la densidad y presión de nuestro disolvente para que coincida con lo que esperaríamos para el modelo de agua SPC/E. Por lo tanto, en este caso utilizaremos un conjunto en el que el número de átomos, la presión y la temperatura del sistema permanecen constantes: **el conjunto NPT**.

  Realizaremos los mismos pasos que hicimos antes: descargar un archivo de instrucciones de dinámica molecular (MD) y luego utilizar `grompp` y `mdrun` para ejecutar la simulación de MD. Una gran diferencia entre esta fase de simulación y las anteriores es que emplearemos un **archivo de estado de punto de control** (nvt.cpt), que permite una continuación numérica de nuestras simulaciones. Este archivo constituye un punto de control de la simulación NVT anterior, que contiene la energía potencial del sistema y las velocidades.


In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/npt.mdp

# Utilizando grompp para preparar nuestra dinámica molecular de equilibración NPT
source /content/gromacs-2023/bin/GMXRC
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
%%time
%%bash
# Ejecutar nuestra dinámica molecular de equilibración NPT
source /content/gromacs-2023/bin/GMXRC
gmx mdrun -deffnm npt -nb gpu

**⚠️PRECAUCIÓN:** Utilizamos un barostato de Parrinello-Rahman en lugar de Berendsen para la equilibración NPT. Esto no debería ser un problema si el sistema está adecuadamente equilibrado; de lo contrario, la combinación de restricciones y Parrinello-Rahman sería inapropiada, ya que podría conducir a inestabilidades.


😱 **¡COPIA DE SEGURIDAD DE EMERGENCIA!** ¿Te quedaste sin tiempo y no pudiste realizar la equilibración NPT de tu sistema? Utiliza la siguiente celda de código para descargar un sistema ya equilibrado a presión:


In [ ]:
#Usar en caso de emergencia
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab07/6wzu_npt.tar.gz
!tar xzf 6wzu_npt.tar.gz

7. Dado que estamos utilizando un conjunto NPT para mantener nuestra simulación a una presión y densidad constantes, deberíamos verificar si se logra esto.


In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

# Este es un truco para proporcionar opciones interactivas a gmx
echo "Pressure" > options
echo "Density" >> options
echo " "

# Utilizando energy para extraer la presión y densidad del sistema durante la equilibración NPT
gmx energy -f npt.edr -o npt_press_dens.xvg -xvg none < options

In [ ]:
# Graficando la densidad del sistema
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

# Leyendo el archivo de texto que contiene esta información
data = np.loadtxt('npt_press_dens.xvg')

plt.title('Presión durante 0.1 ns de Equilibración (NPT)')
plt.xlabel('Tiempo (ps)')
plt.ylabel('Presión [bar]')
plt.ylim(-250,250)

# Suavizado utilizando Savitzky-Golay
from scipy.signal import savgol_filter
yhat = savgol_filter(data[:,1], 21, 5)

# Graficar datos brutos e interpolación de splines
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.plot(data[:,0], yhat, linestyle='solid', linewidth='2', color='blue')
plt.show()

In [ ]:
# Graficando la presión del sistema
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

# Leyendo el archivo de texto que contiene esta información
data = np.loadtxt('npt_press_dens.xvg')

plt.title('Pressure during 0.1 ns Equilibration (NPT)')
plt.xlabel('Time (ps)')
plt.ylabel('Density [kg•m$^{-3}$]')
plt.ylim(1000,1020)
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

**PREGUNTA❓:** ¿Cuál es el valor promedio de la densidad a lo largo de la trayectoria? ¿Te suena familiar?

Una vez que se complete esta sección, podemos pasar a las simulaciones de MD de producción.

# Parte III – Obtener una ejecución de MD de producción y analizar los resultados


Ahora estamos listos para ejecutar nuestras simulaciones de dinámica molecular (MD). Si estamos utilizando un conjunto NPT típico, simplemente podemos continuar nuestra simulación de manera similar a nuestra segunda equilibración, pero sin tener restricciones de posición (sin la opción `-r`).

1. Si has prestado atención a nuestras ejecuciones de MD anteriores, comprenderás completamente lo que se anota en las siguientes celdas de código. Debido a limitaciones de tiempo y al tamaño del sistema, solo generaremos **0.1 ns de ejecuciones de producción**, mientras que encontrarás que las simulaciones en artículos actuales a menudo corresponden a cientos de ns.


In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/md.mdp

# Utilizando grompp para preparar nuestra simulación de MD de producción
source /content/gromacs-2023/bin/GMXRC
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_1.tpr

In [ ]:
# Verificar el contenido del archivo de MD de producción
!cat md.mdp

In [ ]:
%%time
%%bash
# Ejecutar nuestra simulación de MD de producción
source /content/gromacs-2023/bin/GMXRC
gmx mdrun -deffnm md_1 -nb gpu

2. Para finalizar, visualizaremos nuestra simulación. Para ello, utilizaremos el módulo `trjconv` para extraer solo la proteína de nuestro sistema y convertir nuestra trayectoria en un archivo PDB.

Dado que nuestra proteína puede haberse desplazado desde el centro de la caja y llegar a sus bordes, también aprovecharemos la oportunidad para recentrar la caja periódica en nuestra proteína de modo que sus átomos no se extiendan a través de los bordes de las condiciones de contorno periódicas mediante las opciones `-pbc mol` y `-center`.

In [ ]:
%%bash
source /content/gromacs-2023/bin/GMXRC

# Este es un truco para proporcionar opciones interactivas a gmx
echo "Protein" > options
echo "Protein" >> options
echo " "

# Utilizando trjconv para extraer solo los átomos de la proteína de la trayectoria de simulación
# y también recentrar la proteína si sus átomos cruzaron los límites periódicos
gmx trjconv -s md_1.tpr -f md_1.xtc -o md_1_protPBC.pdb -pbc mol -center < options

3. Ahora, puedes descargar este nuevo archivo PDB y cargarlo en [**NGLviewer**](http://nglviewer.org/ngl/) como un archivo PDB de **trayectoria** para visualizar los movimientos de la proteína explorados en este breve tiempo de simulación.

😱 **COPIA DE SEGURIDAD DE EMERGENCIA:** Si no obtuviste la trayectoria de MD de producción a tiempo, puedes descargar <a href="https://github.com/pb3lab/ibm3202/blob/master/files/emergency_backup/lab07/md_1_protPBC.pdb" target="_blank"/>esta trayectoria de 0.2 ns</a> haciendo clic con el botón derecho en *View Raw* y seleccionando la opción de descargar como archivo.

**¡Y este es el final del séptimo tutorial!** En el próximo tutorial, aprenderemos cómo realizar algunos análisis de nuestras ejecuciones de MD de producción.

Si deseas descargar tus resultados, puedes realizar los siguientes comandos:

In [ ]:
os.chdir("/content/")
!zip -r md.zip $mdpath
from google.colab import files
files.download("/content/md.zip")
os.chdir(mdpath)

**📚TAREAS:** ¡Una vez que hayas llegado hasta aquí, puedes realizar varios ejercicios!

1. Intenta realizar la equilibración sin minimizar el sistema de simulación (es decir, omite el uso de em.mdp y pasa directamente a la equilibración).
2. Cambia el paso de tiempo de 2 fs a 4 fs o 5 fs y observa qué sucede con el sistema de simulación en términos de estabilidad. Si recibes algunas advertencias y no se compila el archivo .tpr, puedes agregar la opción `-maxwarn X`, donde `X` es el número de advertencias de GROMACS que ignorarás.

Con estos ejemplos, será posible entender mejor por qué definir el paso de tiempo adecuado y minimizar y equilibrar adecuadamente el sistema son pasos importantes en todas las simulaciones de MD.

3. Ejecuta tu simulación durante más tiempo (por ejemplo, 1 ns; ~ 1 h en Google Colab).
